In [1]:
from sklearn.model_selection import KFold
import pickle
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from experiments.classifier import GraphClassifierCustomizable
from experiments.train import train, test, cross_validate_given_id_folds
from torch.optim import Adam
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
dataset_name = "ENZYMES"
path = "./data/tmp/ENZYMES"
original_dataset = TUDataset(root=path, name=dataset_name, use_node_attr=True)

# Function to add equal edge weights to a graph
def add_edge_weights(data):
    edge_index = data.edge_index
    edge_weight = torch.ones(edge_index.shape[1], dtype=torch.float)
    return Data(edge_index=edge_index, x=data.x, y=data.y, edge_weight=edge_weight)

# Create a new dataset with added edge weights
dataset = [add_edge_weights(data) for data in original_dataset]
n_input = dataset[0].x.size(1)

# Split the dataset into train and test sets
# train_size = int(0.8 * len(dataset))  # 80% of the dataset for training
# test_size = len(dataset) - train_size

# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for train and test sets
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [2]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
id_folds = list(kf.split(dataset))
with open("id_folds.pkl", "wb") as f:
    pickle.dump(id_folds, f)

In [3]:
with open("id_folds.pkl", "rb") as f:
    loaded_list = pickle.load(f)

In [4]:
ts = torch.linspace(0, 1, 20, dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
# model = GraphClassifierTimeConv(pde='heat', ts=ts, n_input=n_input, n_hidden=16, n_output=6, n_time_conv=5, n_time_pool=5, device=device).to(device)
model = GraphClassifierCustomizable(
    pde='heat', 
    ts=ts, 
    n_input=n_input, 
    n_hidden=16, 
    n_output=6, 
    device=device,
    num_layers=2,
    num_lin_layers_between_pde=1,
    num_lin_layers_after_pde=2,
    p_dropout=0.5
).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# CV
results_heat = cross_validate_given_id_folds(
    model, 
    dataset, 
    id_folds=id_folds, 
    batch_size=2048, 
    num_epochs=100, 
    lr=1e-2, 
    weight_decay=0., 
    device=device
)

In [5]:
ts = torch.linspace(0., torch.pi, 20, dtype=torch.float, device=device)
# Create your model, optimizer, and criterion
model = GraphClassifierCustomizable(
    pde='wave', 
    ts=ts, 
    n_input=n_input, 
    n_hidden=16, 
    n_output=6, 
    device=device,
    num_layers=2,
    num_lin_layers_between_pde=1,
    num_lin_layers_after_pde=2,
    p_dropout=0.5
).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()

# CV
results_wave = cross_validate_given_id_folds(
    model, 
    dataset, 
    id_folds=id_folds, 
    batch_size=2048, 
    num_epochs=100, 
    lr=1e-2, 
    weight_decay=0., 
    device=device
)

In [6]:
results_heat

,fold,train_loss,train_accuracy,validation_loss,validation_accuracy,training_time
0,0,1.726080,0.216667,1.750370,0.175000,16.799571
1,1,1.708893,0.227083,1.789707,0.150000,16.420127
2,2,1.729184,0.231250,1.731558,0.233333,16.131822
3,3,1.718833,0.254167,1.747601,0.225000,16.046622
4,4,1.715832,0.229167,1.685867,0.225000,16.286275


In [7]:
results_wave

,fold,train_loss,train_accuracy,validation_loss,validation_accuracy,training_time
0,0,1.791185,0.172917,1.797022,0.141667,30.691364
1,1,1.791186,0.172917,1.796897,0.141667,33.329144
2,2,1.789785,0.181250,1.809584,0.108333,31.806103
3,3,1.790590,0.179167,1.797667,0.116667,30.836359
4,4,1.791133,0.172917,1.797420,0.141667,30.533021
